In [1]:
from time import time 

import pandas as pd
import numpy as np
import cv2
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skimage import filters

from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

In [4]:
class_mapper = {
    'c0': 'Both hands on wheel',
    'c1': 'Using cell (right)',
    'c2': 'Talking cell (right)',
    'c3': 'Using cell (left)',
    'c4': 'Talking cell (left)',
    'c5': 'Adjusting radio',
    'c6': 'Drinking coffee',
    'c7': 'Reaching back',
    'c8': 'Touching head/neck',
    'c9': 'Eyes off the road',
}

In [44]:
image_list = pd.read_csv('./stat442project/driver_imgs_list.csv')
#train_subject_subset = image_list.subject.value_counts().index.values
#test_subject_subset = image_list.subject.value_counts()[:-9].index.values
#train_image_list = image_list[image_list.subject.isin(train_subject_subset)].reset_index(drop=True)
#test_image_list = image_list[image_list.subject.isin(test_subject_subset)].reset_index(drop=True)


In [6]:
h, w = 96,128
train_greyscale_image_matrix = np.load('train_greyscale.npy')
"""
train_greyscale_image_matrix = np.zeros((image_list.shape[0], h*w), dtype=np.uint8)
nitems = image_list.shape[0]
for i, n in image_list.iterrows():
    img_path = './resized_train/' + n.classname + '/' + n.img
    img = cv2.imread(img_path, 0) # Read as greyscale
    train_greyscale_image_matrix[i, :] = img.reshape(h*w)
    if (i % 500 == 0):
        print('Completed {} of {} images ({} %).'.format(i+1, nitems, (i+1)*100/nitems))
"""

"\ntrain_greyscale_image_matrix = np.zeros((image_list.shape[0], h*w), dtype=np.uint8)\nnitems = image_list.shape[0]\nfor i, n in image_list.iterrows():\n    img_path = './resized_train/' + n.classname + '/' + n.img\n    img = cv2.imread(img_path, 0) # Read as greyscale\n    train_greyscale_image_matrix[i, :] = img.reshape(h*w)\n    if (i % 500 == 0):\n        print('Completed {} of {} images ({} %).'.format(i+1, nitems, (i+1)*100/nitems))\n"

In [7]:
test_greyscale_image_matrix = np.load('test_greyscale.npy')
"""
test_images = !ls resized_test/
n_items = len(test_images)
test_greyscale_image_matrix = np.zeros((n_items, h*w), dtype=np.uint8)

for i, n in enumerate(test_images):
    img_path = './resized_test/' + n
    img = cv2.imread(img_path, 0) # Read as greyscale
    test_greyscale_image_matrix[i, :] = img.reshape(h*w)
    if (i % 500 == 0):
        print('Completed {} of {} images ({} %).'.format(i+1, nitems, (i+1)*100/nitems))
"""

"\ntest_images = !ls resized_test/\nn_items = len(test_images)\ntest_greyscale_image_matrix = np.zeros((n_items, h*w), dtype=np.uint8)\n\nfor i, n in enumerate(test_images):\n    img_path = './resized_test/' + n\n    img = cv2.imread(img_path, 0) # Read as greyscale\n    test_greyscale_image_matrix[i, :] = img.reshape(h*w)\n    if (i % 500 == 0):\n        print('Completed {} of {} images ({} %).'.format(i+1, nitems, (i+1)*100/nitems))\n"

In [8]:
X_train = train_greyscale_image_matrix # Transpose if not loading .npy array
y_train = image_list.classname
X_test = test_greyscale_image_matrix # Transpose if not loading .npy array

In [9]:
from sklearn.decomposition import PCA

In [9]:
pca = PCA(n_components=300)

In [10]:
#t0 = time()
#pca.fit(X_train)
#print("done in {}s".format(time() - t0))

pca = pickle.load(open('final_pca_model.sav', 'rb'))

In [11]:
filename = 'final_pca_model.sav'
pickle.dump(pca, open(filename, 'wb'))

In [13]:
t0 = time()
X_train_PCA = pca.transform(X_train)
print("done in {}s".format(time() - t0))


done in 320.841305017s


In [16]:
np.save('X_train_PCA', X_train_PCA)

In [17]:
#t0 = time()
#X_test_PCA = pca.transform(X_test)
#print("done in {}s".format(time() - t0))

X_test_PCA = np.load('X_test_PCA.npy')


In [18]:
np.save('X_test_PCA', X_test_PCA)

In [19]:
neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=4)

In [20]:
t0 = time()
neigh.fit(X_train_PCA, y_train)
print("done in {}s".format(time() - t0))

done in 0.473508834839s


In [ ]:
filename = 'final_5nn_model.sav'
pickle.dump(neigh, open(filename, 'wb'))

In [21]:
t0 = time()
probabilities = neigh.predict_proba(X_test_PCA)
print("done in {}s".format(time() - t0))

done in 451.90060091s


In [ ]:
np.save('PCA_probabilities_5nn.np', probabilities)

In [22]:
np.savetxt("PCA_submission.csv", probabilities, delimiter=",", header="img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9")


In [1]:
## Visualizing principal components

comp = pca.components_[4,:].reshape(h,w)
cv2.imshow('image', cv2.resize(comp, dsize=(w*5,h*5))*100)
cv2.waitKey(0)


NameError: name 'pca' is not defined